In [3]:
import gzip
import datetime
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Input, Dense, Masking
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from biomart import BiomartServer

2024-06-09 10:10:50.967526: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
with gzip.open('../data/GSE116222_Expression_matrix.txt.gz', 'rt') as f:
    expr_matrix = pd.read_csv(f, sep='\t', index_col=0)

In [4]:
expression_data = expr_matrix.T

labels = expression_data.index.str.split('-').str[1].to_list()

inflammation_labels = ['UC inflamed' if x.endswith('3') else 'UC non-inflamed' if x.endswith('2') else 'Healthy' for x in labels]
inflammation_labels = pd.Series(inflammation_labels, index=expression_data.index, name='Inflammation')

In [5]:
server = BiomartServer("http://www.ensembl.org/biomart")
mart = server.datasets['hsapiens_gene_ensembl']

response = mart.search({
    'filters': {
        'biotype': 'protein_coding'
    },
    'attributes': [
        'external_gene_name'
    ]
})

In [6]:
protein_coding_genes = [line.strip() for line in response.iter_lines(decode_unicode=True)]
protein_coding_genes = set(protein_coding_genes)

In [7]:
filtered_df = expression_data[[gene for gene in expression_data.columns if gene in protein_coding_genes]]

In [8]:
# drop columns with zero variance
filtered_df = filtered_df.loc[:, filtered_df.var() > 0]

In [16]:
dataset = filtered_df.iloc[:10, :10]

dataset

,SAMD11,NOC2L,KLHL17,PLEKHN1,PERM1,HES4,ISG15,AGRN,RNF223,C1orf159
AAACCTGGTAATCGTC-A1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.515781,0.0,0.0,0.0
AAACGGGAGCTTTGGT-A1,0.0,0.000000,0.0,0.0,0.0,1.762574,1.227800,0.0,0.0,0.0
AAACGGGTCTGGTATG-A1,0.0,0.000000,0.0,0.0,0.0,0.000000,1.694233,0.0,0.0,0.0
AAAGTAGAGAACTGTA-A1,0.0,1.367324,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
AAAGTAGCAGGGAGAG-A1,0.0,0.000000,0.0,0.0,0.0,0.000000,1.574027,0.0,0.0,0.0
AAAGTAGGTTCGTTGA-A1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
AAAGTAGTCGGGAGTA-A1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
AAATGCCCAAGACGTG-A1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.748445,0.0,0.0,0.0
AAATGCCTCCTTTCTC-A1,0.0,0.856897,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
AACACGTAGAGCAATT-A1,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [4]:
dataset = pd.DataFrame(np.random.rand(8, 4))  # 8 samples, 10 genes

# Define the autoencoder architecture
input_dim = dataset.shape[1]  # Number of genes
encoding_dim = 2  # Dimension of the encoded representation
epochs = 5  # Maximum number of epochs
batch_size = 2 # Batch size for training
mask_value = -np.inf  # Value to use for masking

input_layer = Input(shape=(input_dim,))
masked_input = Masking(mask_value=mask_value)(input_layer)
encoded = Dense(encoding_dim, activation='relu')(masked_input)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = Model(input_layer, decoded)

# Print model summary
autoencoder.summary()

def custom_loss(y_true, y_pred):
    mask = tf.not_equal(y_true, mask_value)
    mask = tf.cast(mask, dtype=tf.float32)
    loss = tf.reduce_sum(mask * tf.square(y_true - y_pred), axis=-1) / tf.reduce_sum(mask, axis=-1)
    return loss

# Compile the model with the custom loss function
autoencoder.compile(optimizer='adam', loss=custom_loss)

# Function to create masked input
def create_masked_input(data, mask_index, mask_value):
    masked_data = np.copy(data)
    masked_data[:, mask_index] = mask_value
    return masked_data

# Prepare training data
train_data = dataset.sample(frac=0.8, random_state=42)
test_data = dataset.drop(train_data.index).values
train_data = train_data.values

# Print input shapes
print("Shape of train_data:", train_data.shape)
print("Shape of test_data:", test_data.shape)

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with masking
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    for i in range(input_dim):
        masked_input = create_masked_input(train_data, i, mask_value)
        autoencoder.fit(masked_input, train_data, 
                        epochs=1,
                        batch_size=batch_size, 
                        validation_data=(create_masked_input(test_data, i, mask_value), test_data), 
                        callbacks=[early_stopping],
                        verbose=1)

# Predicting the expression of a specific gene
def predict_gene_expression(data, gene_index, mask_value):
    masked_input = create_masked_input(data, gene_index, mask_value)
    predictions = autoencoder.predict(masked_input)
    return predictions[:, gene_index]

# Example usage
gene_index = 0  # Index of the gene to predict
test_sample = test_data[0].reshape(1, -1)  # Reshape to match input shape
predicted_expression = predict_gene_expression(test_sample, gene_index, mask_value)
print(predicted_expression)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ masking (Masking)               │ (None, 4)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │            12 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22 (88.00 B)

 Trainable params: 22 (88.00 B)

 Non-trainable params: 0 (0.00 B)

Shape of train_data: (6, 4)
Shape of test_data: (2, 4)
Epoch 1/5
